# Eliminación de preguntas repetidas
Ya con todas las preguntas que cumplen los requisitos del INE para ser tomadas en cuenta, vamos 
a hacer otra depuración para eliminar aquellas preguntas que están repetidas.
Es importante considerar que dos preguntas pueden ser iguales aún cuando estén formuladas con 
palabras diferentes (es decir, una puede ser paráfrasis de la otra).
El resultado de este paso serán dos archivos:

- Uno que contenga el texto de las preguntas repetidas, así como una métrica que mida su parecido. De estas preguntas, sólo se conservará 1 en el dataset de trabajo.

  - La pregunta que será conservada es aquella que fue enviada primero de acuerdo al timestamp
          
- Un archivo con las preguntas depuradas (eliminando las repeticiones)

In [2]:
!pip install spacy
!pip install scikit-learn
!python -m spacy download es_core_news_md

     ---------------------------------------- 0.0/42.3 MB ? eta -:--:--
     --------------------------------------- 0.0/42.3 MB 217.9 kB/s eta 0:03:14
     --------------------------------------- 0.0/42.3 MB 393.8 kB/s eta 0:01:48
     --------------------------------------- 0.1/42.3 MB 726.2 kB/s eta 0:00:59
     ---------------------------------------- 0.3/42.3 MB 1.6 MB/s eta 0:00:27
      --------------------------------------- 0.6/42.3 MB 2.6 MB/s eta 0:00:16
     - -------------------------------------- 1.2/42.3 MB 4.3 MB/s eta 0:00:10
     -- ------------------------------------- 2.4/42.3 MB 7.3 MB/s eta 0:00:06
     --- ------------------------------------ 3.7/42.3 MB 10.3 MB/s eta 0:00:04
     ---- ----------------------------------- 4.9/42.3 MB 12.0 MB/s eta 0:00:04
     ----- ---------------------------------- 6.3/42.3 MB 13.9 MB/s eta 0:00:03
     ------- -------------------------------- 7.7/42.3 MB 15.4 MB/s eta 0:00:03
     -------- ------------------------------- 9.1/42

In [3]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity

limpio_df = pd.read_csv('./Base_limpia.csv')

nlp = spacy.load('es_core_news_md')

preguntas = limpio_df['pregunta_norm']

vectores = [nlp(str(pregunta)).vector for pregunta in preguntas]

similitud = cosine_similarity(vectores)

repetidas = []

for index, fila in limpio_df.iterrows():
    for j in range(index + 1, len(preguntas)):
        if similitud[index, j] > 0.9:
            p1_fecha = limpio_df.loc[index]['fecha']
            p2_fecha = limpio_df.loc[j]['fecha']
            if p1_fecha < p2_fecha:
              repetidas.append(pd.DataFrame([[limpio_df.iloc[j]['pregunta'], limpio_df.loc[index]['pregunta_norm']]], index=None))
            else:
              repetidas.append(pd.DataFrame([[limpio_df.loc[index]['pregunta'], limpio_df.iloc[j]['pregunta_norm']]], index=None))
repetidas = pd.concat(repetidas)
repetidas.columns = ['repetida_de', 'pregunta']

display(repetidas)
repetidas.to_csv('./preguntas_repetidas.csv', index=False)

limpio_df = limpio_df[limpio_df['pregunta_norm'].isin(repetidas['pregunta']) == False]
display(limpio_df)
limpio_df.to_csv('./Base_sin_repetidas.csv', index=False)

,repetida_de,pregunta
0,En México muchas personas se quejan del sistem...,como podria usted cambiar sistema salud public...
0,En México muchas personas se quejan del sistem...,como podrias incrementar eficacia servicios sa...
0,En México muchas personas se quejan del sistem...,digan opinion respecto servicio salud mejor di...
0,En México muchas personas se quejan del sistem...,sistema salud insuficiente pregunta como podem...
0,En México muchas personas se quejan del sistem...,cual plan concreto mejorar acceso atencion med...
...,...,...
0,¿Cuál sería su enfoque para abordar la violenc...,candidato presidencial cuales propuestas comba...
0,¿Cómo abordará el problema de la desigualdad e...,como abordara brecha educativa zonas urbanas r...
0,¿Cuál es su enfoque para abordar la violencia ...,cual estrategia proponen prevenir violencia ge...
0,"¿Cuáles son sus propuestas para concientizar, ...",estrategias implementaran sensibilizar socieda...


,entidad,edad,genero,grupo_discriminacion,tema,pregunta,fecha,pregunta_norm
0,Ciudad de México,43,Femenino,NaN,No discriminación y grupos vulnerables,"Candidat@, cómo apoyaría a los productores de ...",2024-03-22 00:04:00,candidat como apoyaria productores nopal alcal...
1,Aguascalientes,14,Masculino,NaN,Salud,En México muchas personas se quejan del sistem...,2024-03-22 00:04:00,mexico muchas personas quejan sistema salud co...
2,Nuevo León,17,Masculino,Personas de la diversidad sexual,Educación,¿Qué estrategias y planes concretos se propone...,2024-03-22 00:02:00,estrategias planes concretos proponen asegurar...
4,México,58,Masculino,Personas con discapacidad,Combate a la corrupción,"Por qué ,sigue la corrupción a pesar se, saber...",2024-03-21 23:59:00,sigue corrupcion pesar saber donde si origuina
5,Guanajuato,40,Masculino,NaN,Transparencia,Quisiera participar en estás elecciones,2024-03-21 23:58:00,quisiera participar elecciones
...,...,...,...,...,...,...,...,...
22884,Baja California,68,Masculino,NaN,Salud,Qué porcentaje del Presupuesto de Egresos de l...,2024-02-20 20:06:00,porcentaje presupuesto egresos federacion dest...
22885,Morelos,69,Masculino,Personas adultas mayores,Transparencia,"La administracion y control, de puertos, aerop...",2024-02-20 19:47:00,administracion control puertos aeropuertos via...
22886,Sonora,44,Femenino,Personas afromexicanas,No discriminación y grupos vulnerables,Que hará para poder favorecer a los mexicanos ...,2024-02-20 12:27:00,hara poder favorecer mexicanos vulnerables alg...
22887,Guanajuato,25,Masculino,NaN,Violencia en contra de las mujeres,Se le cerraran las puertas a las mujeres en pa...,2024-02-20 11:59:00,cerraran puertas mujeres palacio gobierno
